In [1]:
# 필요한 package 
import pandas as pd 
import ast #str to list
from tqdm import tqdm 
import tqdm
import numpy as np
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)
import re
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunkeun_jo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def read_data():
    hyatt= pd.read_csv("hyatt_comment",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_comment",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_comment",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham_comment",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom,comment_token,final
0,1,"['❤️❤️❤️', '👏', '🤤❤️', 'OMG WHAT THIS IS AWESO...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[['@allpurple10 💙🍪'], [""@hamptonrockyhill we'r...","[['❤️❤️❤️'], ['👏'], ['🤤❤️'], ['OMG', 'WHAT', '...","['❤️❤️❤️ @allpurple10 💙🍪', ""👏 @hamptonrockyh..."
1,2,"['This is awesome!!!👏👏👏', 'Thank you!!', '👏🏻👏🏻...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[['@allpurple10 thank you!'], [], [], [], [], ...","[['This', 'is', 'awesome', '!', '!', '!', '👏👏👏...",['This is awesome!!!👏👏👏 @allpurple10 thank yo...
2,3,"['Build with immortal love 👑', '❤️', '\U0001f9...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], ['@erattray1 I saw that earli...","[['Build', 'with', 'immortal', 'love', '👑'], [...","['Build with immortal love 👑 ', '❤️ ', '\U0001..."
3,4,['.\n.\n.\n.\n#staugustine #staugustinefl #sta...,[0],[4284913372],[[]],"[['.', '.', '.', '.', '#', 'staugustine', '#',...",['.\n.\n.\n.\n#staugustine #staugustinefl #sta...
4,5,['.\n.\n.\n.\n#boca #bocaraton #resortlife #ex...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], ['@patriciatorressl Waldorf?', '@giles.yn...","[['.', '.', '.', '.', '#', 'boca', '#', 'bocar...",['.\n.\n.\n.\n#boca #bocaraton #resortlife #ex...


In [5]:
# 항상 제일 먼저 해줘야 합니다. 
def string_to_list(df,columns):
    for col in columns:
        df[col]=df[col].apply(lambda x: ast.literal_eval(x))
        
for hotel in (hilton,hyatt,holiday,wyndham):
    string_to_list(hotel,["comment","comment_like","comment_writer","new_recom","comment_token","final"])
hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom,comment_token,final
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[[❤️❤️❤️], [👏], [🤤❤️], [OMG, WHAT, THIS, IS, A...","[❤️❤️❤️ @allpurple10 💙🍪, 👏 @hamptonrockyhill..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[[This, is, awesome, !, !, !, 👏👏👏], [Thank, yo...",[This is awesome!!!👏👏👏 @allpurple10 thank you...
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]...","[Build with immortal love 👑 , ❤️ , 🧡🧡🧡 , sooo ..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],"[[., ., ., ., #, staugustine, #, staugustinefl...",[.\n.\n.\n.\n#staugustine #staugustinefl #stau...
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[[., ., ., ., #, boca, #, bocaraton, #, resort...",[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...


이제 comment와 recomment를 합친 피드와 내용의 tokenize를 해서 저장한 다음에 나머지 내용은 지우도록 하겠습니다.

In [7]:
# comment에 대한 tokenize
def cleaned_text(text):
    # 누구를 tag한 태그 표시는 지우기
    #전의 text
    prev_text=text
    while True:
        text=re.sub("@[\d\w]+","",text)
        
        if prev_text!=text:
            prev_text=text
        else:
            break
            
    return text

def text_tokenize(text):
    return [word_tokenize(cleaned_text(sentence)) for sentence in text]

for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["comment_token"]=hotel["final"].apply(lambda x: text_tokenize(x))
    
hilton.head()

# comment에 대한 토큰화 작업 끝 

,post_id,comment,comment_like,comment_writer,new_recom,comment_token,final
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[[❤️❤️❤️, 💙🍪], [👏, we, 're, pretty, excited, a...","[❤️❤️❤️ @allpurple10 💙🍪, 👏 @hamptonrockyhill..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[[This, is, awesome, !, !, !, 👏👏👏, thank, you,...",[This is awesome!!!👏👏👏 @allpurple10 thank you...
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]...","[Build with immortal love 👑 , ❤️ , 🧡🧡🧡 , sooo ..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],"[[., ., ., ., #, staugustine, #, staugustinefl...",[.\n.\n.\n.\n#staugustine #staugustinefl #stau...
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[[., ., ., ., #, boca, #, bocaraton, #, resort...",[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...


원하던대로 comment에서 사용된 tokenize를 성공하였습니다. 이제 나머지는 뺀 다음에 정리하도록 하겠습니다.

In [8]:
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel.drop(["comment","comment_like","comment_writer","new_recom","final"],axis=1,inplace=True)
hilton.head()

,post_id,comment_token
0,1,"[[❤️❤️❤️, 💙🍪], [👏, we, 're, pretty, excited, a..."
1,2,"[[This, is, awesome, !, !, !, 👏👏👏, thank, you,..."
2,3,"[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]..."
3,4,"[[., ., ., ., #, staugustine, #, staugustinefl..."
4,5,"[[., ., ., ., #, boca, #, bocaraton, #, resort..."


In [9]:
# 저장하는 함수 
def save_data(hyatt,hilton,holiday,wyndham):
    hyatt.to_csv("hyatt_comment_final",index=False)
    hilton.to_csv("hilton_comment_final",index=False)
    holiday.to_csv("holiday_comment_final",index=False)
    wyndham.to_csv("wyndham_comment_final",index=False)
save_data(hyatt,hilton,holiday,wyndham)

## caption에 대한 tokenizer도 필요할 것이므로 caption까지 진행하겠습니다.

In [2]:
def read_data():
    hyatt= pd.read_csv("hyatt",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_post",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_inn",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()

,date,caption,id,profile,type,caption_and_hashtags,is_video,view_count,video_duration,likes,comments,com_list,caption_tag,real_ugc
0,2020-04-09 16:33:28,"Over the past few weeks, we’ve gotten lots of ...",1,hilton,GraphVideo,[],True,1730.0,30.3,194,29,"[['❤️❤️❤️', 0, '<Profile allpurple10 (31050709...",['@doubletree)'],False
1,2020-04-06 12:37:22,One million ‘thank-yous’ to the frontline medi...,2,hilton,GraphImage,['hotelsforheroes'],False,NaN,NaN,412,19,"[['This is awesome!!!👏👏👏', 0, '<Profile allpur...",['@AmericanExpress'],False
2,2020-04-03 15:31:01,We’re joining with hotels across the world to ...,3,hilton,GraphSidecar,['hope'],False,NaN,NaN,418,7,"[['Build with immortal love 👑', 0, '<Profile l...",[],False
3,2020-03-11 15:54:11,We tend to lose track of things during a stay ...,4,hilton,GraphImage,[],False,NaN,NaN,98,1,[['.\n.\n.\n.\n#staugustine #staugustinefl #st...,['@embassysuitesstaugustinebeach:'],False
4,2020-03-09 18:02:10,"From Lake Boca to the beach, @bocaresort will ...",5,hilton,GraphImage,[],False,NaN,NaN,227,7,[['.\n.\n.\n.\n#boca #bocaraton #resortlife #e...,['@bocaresort'],False


In [3]:
# com_list가 필요 없습니다.
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel.drop(["com_list"],axis=1,inplace=True)

In [36]:
# caption tokenizer
def cleaned_text(text):
    # 누구를 tag한 태그 표시는 지우기
    #전의 text
    prev_text=text
    while True:
        text=re.sub("@[\d\w]+|📷:","",text)
        
        if prev_text!=text:
            prev_text=text
        else:
            break
            
    return text

def text_tokenize(text):
    text=str(text)
    return word_tokenize(cleaned_text(text)) 

for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["caption_token"]=hotel["caption"].apply(lambda x: text_tokenize(x))
    
# hilton.head()

In [41]:
hilton

,date,caption,id,profile,type,caption_and_hashtags,is_video,view_count,video_duration,likes,comments,caption_tag,real_ugc,caption_token
0,2020-04-09 16:33:28,"Over the past few weeks, we’ve gotten lots of ...",1,hilton,GraphVideo,[],True,1730.0,30.3,194,29,['@doubletree)'],False,"[Over, the, past, few, weeks, ,, we, ’, ve, go..."
1,2020-04-06 12:37:22,One million ‘thank-yous’ to the frontline medi...,2,hilton,GraphImage,['hotelsforheroes'],False,NaN,NaN,412,19,['@AmericanExpress'],False,"[One, million, ‘, thank-yous, ’, to, the, fron..."
2,2020-04-03 15:31:01,We’re joining with hotels across the world to ...,3,hilton,GraphSidecar,['hope'],False,NaN,NaN,418,7,[],False,"[We, ’, re, joining, with, hotels, across, the..."
3,2020-03-11 15:54:11,We tend to lose track of things during a stay ...,4,hilton,GraphImage,[],False,NaN,NaN,98,1,['@embassysuitesstaugustinebeach:'],False,"[We, tend, to, lose, track, of, things, during..."
4,2020-03-09 18:02:10,"From Lake Boca to the beach, @bocaresort will ...",5,hilton,GraphImage,[],False,NaN,NaN,227,7,['@bocaresort'],False,"[From, Lake, Boca, to, the, beach, ,, will, ta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2019-12-11 20:53:09,Need ideas for 2020 travel? Use this as your i...,65,hilton,GraphImage,[],False,NaN,NaN,21052,19,"['@conradbali', '@benmikha']",True,"[Need, ideas, for, 2020, travel, ?, Use, this,..."
65,2019-12-11 20:52:41,Can you spot where Hell’s Kitchen ends and Mid...,66,hilton,GraphImage,[],False,NaN,NaN,1308,3,['@theamazingknight'],True,"[Can, you, spot, where, Hell, ’, s, Kitchen, e..."
66,2019-12-09 21:40:25,The sparkling indoor pool isn’t the only water...,67,hilton,GraphImage,[],False,NaN,NaN,851,2,[],False,"[The, sparkling, indoor, pool, isn, ’, t, the,..."
67,2019-12-09 21:39:43,"Stay adventurous, stay enchanted. The mesmeriz...",68,hilton,GraphImage,[],False,NaN,NaN,13743,22,['@randomlies'],True,"[Stay, adventurous, ,, stay, enchanted, ., The..."


In [42]:
def save_data(hyatt,hilton,holiday,wyndham):
    hyatt.to_csv("hyatt_final",index=False)
    hilton.to_csv("hilton_final",index=False)
    holiday.to_csv("holiday_final",index=False)
    wyndham.to_csv("wyndham_final",index=False)
save_data(hyatt,hilton,holiday,wyndham)

# 이제부터는 final comment 데이터에 대한 sentiment 분석을 하겠습니다.